# 本文主要记录YOLO v3的实现过程

---

## 实现思路
1. 依据数据格式，实现“数据读取”功能；
2. 基础主干网络ResNet-18实现（本项目还包括：resnet18-v2，mixnet，mobilenet-v2，resnext等backbone）；
3. 实现3个head，形成YOLO v3整体模型；
4. YOLO v3 损失函数实现：网络输出解码、label编码、计算损失；
5. 边边角角：配置与训练脚本、测试脚本、预测脚本，等等；
6. 进阶修改：损失函数修改，主干网络修改，等等。

---

## 本项目中值得关注的功能

1. **数据预处理与标签的协同**：由于图片在预处理阶段，会进行维持原图比例进行resize的操作，而标签坐标由于是相对原图的比例，故resize后的标签坐标也要进行变换；
1. `tf.keras.model.fit`函数中**日志打印**，继承了Callback的日志打印类：在总损失的基础上，增加各项损失函数的输出；
1. **自定义TensorBoard**：绘制不同子损失项的曲线、BN层gamma的分布；
1. 每个输入图片对应的目标label不定长，需pad到等长才能使用`tf.data.Dataset`对象构造训练数据集；
1. 本项目的YOLO v3的实现，相比其他TensorFlow版本的实现，**更加接近原论文、Darknet、Caffe版本的实现**；
1. 基于head的灵活设置：不同head可设置不同的参数（anchor数、子损失项权重等）；
1. 新增单一类别检测时，可以设置只有坐标回归项和IOU项，不包含类别损失项；
1. 多尺度训练：输入不同尺度的图片进行训练；
1. focal loss在YOLO v3损失函数中的应用；
1. GHM损失函数在YOLO v3损失函数中的应用；
1. 利用GIOU替换IOU计算；
1. 利用TIOU替换IOU计算；


### 数据预处理与标签的协同

在图片进行原比例resize的同时，对label坐标进行等效变换。

### 日志打印

本项目使用`tf.keras.model.fit`函数进行训练，日志打印类继承了`keras.callbacks.Callback`。由于父类会将总损失函数值传入`tf.keras.model`对象的`Callback`对象，而我还想查看各部分`loss`的情况，所以我还需要自己记录各项损失的值，然后全部打印出来。

实现过程中主要考虑的点有三个：

1. 需要损失函数的细节部分：**前期预矫正所有grid、所有anchor的xywh的`rectified_coord_loss`，中心点偏移损失`coord_loss_xy`，长宽偏差损失`coord_loss_wh`，背景anchors的IOU损失`noobj_iou_loss`，前景anchors的IOU损失`obj_iou_loss`，分类损失`class_loss`，正则项损失`regularization_loss`。**

2. 更新这些细节损失的值：在自定义损失函数类`YOLOv3Loss`的`__init__`函数里，定义了以上各项损失的TensorFlow变量，然后在每次计算总损失时（`loss(pred, target)`函数），更新各项损失的变量的值；

3. 在日志打印类`DetailLossLogger`里，增加这些变量到打印列表中。

以上三点，可以在源码中找到：`yolov3/trainer.py`的`train()`函数，`yolov3/yolov3_loss.py`中的`__init__()`函数和`loss()`函数，`utils/logger_callback.py`。

### 自定义TensorBoard

自定义TensorBoard（`utils/board_callback.py`），可查看：

1. scalar：各个head的子损失函数变化曲线；

1. histogram：网络中所有BN层的所有gamma变量的分布；

### 构造等长label的`tf.data.Dataset`对象


本来是可以分别构造输入图形数据集和标签数据集，然后使用组合拼装，最后拉batch和缓存，实现输入数据集的构造。

``` python
image_set = dataset.map(functools.partial(FileUtil._parse_image, image_size=image_size),
                num_parallel_calls=tf.data.experimental.AUTOTUNE)
labels_set = dataset.map(FileUtil._parse_labels,
                 num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = tf.data.Dataset.zip((image_set, labels_set))
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
```

但是，由于同一批次里的标签数据集中每一个sample的标签pad到等长，所以需要改为：先各自构造batch，然后在拼装，最后缓存，实现输入数据集的构造。

``` python
image_set = image_set.batch(batch_size)
labels_set = labels_set.padded_batch(batch_size, (tf.TensorShape([None])), padding_values=-1.)
dataset = tf.data.Dataset.zip((image_set, labels_set))
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
```

### 遵从原版YOLO v3的实现

一开始我使用Caffe版本YOLO v2 [quhezheng/caffe_yolo_v2](https://github.com/quhezheng/caffe_yolo_v2)，看了部分源码。然后后续为了使用上的灵活，换成TensorFlow版本的。但是看了好多其他人YOLO v2、v3的比较热门的开源库后发现，其中多多少少都有点和原版对不上，然后写了TensorFlow版本的YOLO v2 [zheng-yuwei/YOLOv2-tensorflow](https://github.com/zheng-yuwei/YOLOv2-tensorflow)；后续为了填补YOLO v3在这些问题的空缺，也写了该库。这些对不上的空缺包括：

1. **什么才是`response anchor`?**

原实现中指出，目标中心点所在的`grid`中，所有的预定义`anchors`的预测`bounding box`与目标`bounding box`的IOU最大的`anchor`，为`response anchor`！

原版Darknet的处理方式是：使用`anchor box`和`target box`计算IOU，取出IOU最大的`anchor box`所在的位置，即为该`target box`的`response anchor`。这对于长宽比较靠近某个预定义anchor的目标可能问题不大，但是对于那些长宽在多个预定anchor边缘的目标而言，可能这种硬切分最终不利于训练。

这里的思路基本与原文相匹配，但我觉得还不够好。因为个人觉得，应该使用`predict box`和`target box`计算IOU，取出IOU最大的`predict box`所在的位置，即为该`target box`的`response anchor`。但是这种训练方式可能导致训练前期极其不稳定，甚至训练不下去。所以作者也在Darknet YOLOv2其实提出了一种训练方式，就是前期设置一个矫正损失项，负责使所有`predict boxes`回归各自所属的`anchor`，这一个操作相当于给每个预测赋予了先验信息。（但是Darknet YOLOv3作者并没有使用这个操作）

这里，一般其他TensorFlow/Pytorch库会遵照Darknet做法，使用`anchor box`与`target box`的IOU来获取`response box`，并且也没有前期校正损失的操作。

所以，我个人感觉我是解决了逻辑bug。。。（当然见仁见智了）

2. **损失函数用MSE还是CE？**

因为损失函数的公式，只有YOLO v1的论文写的均方差（MSE）损失，后续的论文没写，而源码中使用的也是MSE损失。那么在我们自己实际训练过程中，对于**分类损失和Score损失，究竟使用均方差（MSE）损失好还是交叉熵（CE）损失呢？**

我在`1_learning_note/Use_Cross_Entropy_or_Mean_Square_Error_Loss.ipynb`中，对比了两者，发现：
1. MSE损失虽然对预测误差是二次关系，但CE损失对预测误差是$[0,1]$间的对数增长关系，更加敏感（也就是loss会更加大）；
2. MSE损失存在在梯度 小-大-小 的波动现象，而CE损失函数的梯度和误差是单调增关系，更有利于对大误差样本的学习。

基于上面对CE损失函数和MSE损失函数的认识，我们先做数值的直观感受：
1. 在与目标值误差为0.1时，MSE损失函数值为0.01，CE损失函数值为log(0.9)=0.046;
2. 在与目标值误差为0.3时，MSE损失函数值为0.09，CE损失函数值为log(0.7)=0.155;

所以如果我们以$[0.1,0.3]$为允许的误差范围，参考原论文各项损失的权重为：前期MSE校正坐标损失权重系数为0.01，MSE坐标损失权重系数为5，MSE背景Score损失权重系数为0.5，MSE前景Score损失权重系数为1，MSE类别损失权重系数为1。故将分类损失和Score损失从MSE损失改为CE损失时，是否可以把除坐标损失外的其他损失项的权重系数，用$[1.5, 5]$范围的除数来调参协调各项损失。当然，别忘了，这里有L2正则化损失项的权重系数。不过感觉可以将YOLOv2损失权重系数和L2正则化损失权重系数作为两个体系大概调整。

在`yolov3/yolov3_loss.py`的`loss()`函数里，分类损失和Score损失实现的是CE损失函数，但是也加了MSE损失的注释。当然，可能最好的方式是：基于理论理解上的实践吧。



最后，`yolov3/yolov3_loss.py`里，**损失函数计算的主要步骤**为：
1. 利用`tf.map_fn`对每个sample逐一计算损失；
2. 计算所有grid、anchor的预测bounding box和实际目标bounding box的最大IOU，作为后续判断该anchor是否需要计算背景损失的依据；
3. 取出实际目标中心所在grid的anchor，计算与对应的目标的IOU，得到最大IOU的anchor的位置，作为后续计算前景损失的依据。

### 基于head的灵活参数设置

- 每个head的anchor数独立：模型输出head和损失函数实现时，并没有限制每个head的anchor数，而是根据`configs.py`文件中的`FLAGS.anchor_boxes`中的数组shape自适应调整。

- 每个head的子损失项权重系数独立设置。

### 单类别检测可以设置不包含类别损失项

实时上，由于YOLO系列算法的前背景是通过IOU来区分的，所以当只有单类别的检测时，是可以舍弃掉类别损失项的。于是，我用`config.py`中包含的`FLAGS.class_num`来标记类别数外，还用于标记是否要包含类别损失项：`FLAGS.class_num=0`表示单类别但不包含类别损失项。此时，YOLO v3网络的输出不包含类别的channel，在做网络输出解码、标签转换、损失计算时，都相应的不会计算类别损失！

### 多尺度训练

**聚类anchor的时候，需要计算归一化的anchor，这样可以使anchor和实际图形尺度解耦。**

由于训练过程中数据预处理的原因，可能使得实际图像尺度变化，如果此时再用这批数据去聚类anchor：
1. 如果是归一化的操作，聚类出来的anchor与预处理前聚类出来的anchor是一致的；
2. 而如果不是归一化的anchor，那么聚类出来的anchor尺度和最后的feature map大小（如$13 \times 13$）是相关的（例如是归一化的anchor*feature map尺寸）;而由于输入尺度的变化会造成输出尺度变化，最终导致聚类得到的预定义anchor和实际训练过程中数据潜在的anchor不一致！

除了多尺度训练时，使用归一化的anchor之外，训练时最好也不要用crop这些改变目标相比原图比例的技巧。

### focal loss在YOLO v3损失函数中的应用
focal loss的提出在于解决前景、背景的样本数不平衡问题（转为为难易样本的平衡问题，难易则用预测偏差 $(p - \hat{p})^\gamma$ 来体现，可能还会给加上$\alpha$ 进行样本数调参控制）。YOLO v3中（复习YOLO v3损失函数可参考[YOLO v2](https://zheng-yuwei.github.io/2018/10/03/4_YOLOv2/)），整体损失函数（这里都是用了MSE）为：

$$
\begin{matrix}
L = \lambda_{coord}*(L_{xy} + L_{wh}) + L_{iou}^{obj} + \lambda_{noobj}*L_{iou}^{noobj} + L_{class} \\
\begin{eqnarray}
& L_{xy} = & \sum_{i=0}^{S^2} \sum_{j=0}^{B} I_{ij}^{obj}[(x_{i}-\hat{x}_{i})^2 + (y_{i}-\hat{y}_{i})^2] \\
& L_{wh} = & \sum_{i=0}^{S^2} \sum_{j=0}^{B} I_{ij}^{obj}[(\sqrt{w_{i}}-\sqrt{\hat{w}_{i}})^2 + (\sqrt{h_{i}}-\sqrt{\hat{h}_{i}})^2] \\
& L_{iou}^{obj} = & \sum_{i=0}^{S^2} \sum_{j=0}^{B} I_{ij}^{obj}(C_{i}-\hat{C}_{i})^2 \\
& L_{iou}^{noobj} = & \sum_{i=0}^{S^2} \sum_{j=0}^{B} I_{ij}^{noobj}(C_{i}-\hat{C}_{i})^2 \\
& L_{class} = & \sum_{i=0}^{S^2} I_{ij}^{obj} \sum_{c=class} (p_{i}(c)-\hat{p}_{i}(c))^2
\end{eqnarray}
\end{matrix}
$$

由于分类损失只基于response anchor计算，故不存在前/背景失衡的问题（虽然也有难易样本失衡的可能性，但这个失衡和前/背景失衡不是一个量级上的事情，可能还没到考虑“失衡”的情况）。而**真正将前/背景区分开来的是noobj的IOU损失和obj的IOU损失**，所以focal loss应该在这两项上使用：

$$
\begin{eqnarray}
& L_{iou}^{obj} = & - \sum_{i=0}^{S^2} \sum_{j=0}^{B} ( I_{ij}^{obj} * log(IOU_{pred})) \\
& L_{iou}^{noobj} = & - \sum_{i=0}^{S^2} \sum_{j=0}^{B} ( I_{ij}^{noobj} * log(1 - IOU_{pred}))
\end{eqnarray}
$$

变为：

$$
\begin{eqnarray}
& L_{iou}^{obj*} = & \alpha_{obj} * |1-IOU_{pred}|^{\gamma} * L_{iou}^{obj} \\
& L_{iou}^{noobj*} = & \alpha_{noobj} * |IOU_{pred}|^{\gamma} * L_{iou}^{noobj}
\end{eqnarray}
$$



### GHM损失函数在YOLO v3损失函数中的应用
待实现，解释可参考[GHM论文理解及实现](https://zheng-yuwei.github.io/2019/07/08/13_GHM%E8%AE%BA%E6%96%87%E7%90%86%E8%A7%A3%E5%8F%8A%E5%AE%9E%E7%8E%B0/)，实现可参考[zheng-yuwei/multi-label-classification](https://github.com/zheng-yuwei/multi-label-classification)。

### 利用GIOU替换IOU计算
待实现

### 利用TIOU-Recall替换IOU计算
计算IOU时，计算的是相交区域占总区域的比例：
$$
IOU = \frac{A(G_{i} \bigcap D_{j})}{A(G_{i} \bigcup D_{j})}
$$
其中，$G_{i}$是ground truth，$D_{j}$是预测框，$A(*)$是面积计算函数。

TIOU-Recall同时考虑了相交区域占ground truth区域的比例，也就是召回率：
$$
TIOU_{Recall} = \frac{A(G_{i} \bigcap D_{j})}{A(G_{i} \bigcup D_{j})} * \frac{A(G_{i} \bigcap D_{j})}{A(G_{i})}
$$

IOU的计算结果主要用在：
1. 所有grids的所有anchors与所有待检测物体中的最大IOU；
1. 所有grids的所有anchors的最大IOU是否大于IOU阈值（是否作为背景）；
1. response grids的所有anchors与对应目标的最大IOU，从而成为response anchors；

若使用TIOU-Recall，会使的IOU的值偏低，所以阈值应该也得调低，同时使得IOU预测误差上升（若目标IOU=1，当然有的实现设定目标IOU=实际IOU）：
1. 背景IOU中最大IOU加上了召回率的考虑；
1. 乘上召回率减低了IOU，使得更低于IOU阈值（若不调整），更有可能成为背景，漏检概率可能会升高；
1. response grids中最大IOU也偏向召回率的考虑，召回率更高的anchors更有可能负责预测该物体；

---

## 待续
YOLO系列的确很棒，YOLO v3的确很厉害，工业上大部分场景还是可以胜任的。但是anchor-based的思想在那，看得见的缺陷。所以想玩玩anchor free的算法。

另外，随着IOT、5G、机器人等技术的兴起，端侧如何做到更小更快更好，需要芯片、框架、NAS等方面的助力；云侧则会从图像分析逐渐演变到视频分析，综合多帧获取更多的信息，提升指标实现商业落地的可能。

所以，我后续可能的方向有：
- anchor free检测：CenterNet；
- 两阶段检测：Faster R-CNN等；
- 关键点检测；
- 视频/移动检测：YOWO；
- 图像分割；
- 图像/视频增强；

实战需扩展知识：
- 蒸馏（用过label-smoothing，有一定用处；而我理解label-smoothing是给定一个普遍性假设先验，蒸馏是针对样本的先验，然后根据先验设置soft label）；
- AutoML，NAS；
